In [1]:
import datetime

class Ticket:
    def __init__(self, ticket_id, issue, priority, status="Unassigned", department=None):
        self.ticket_id = ticket_id
        self.issue = issue
        self.priority = priority
        self.status = status
        self.department = department
        self.creation_time = datetime.datetime.now()
        self.next = None

class TicketSystem:
    def __init__(self):
        self.head = None
        self.ticket_counter = 101
        self.deleted_tickets = []

        self.departments = {
            "Flight Delay": "Support",
            "Baggage Lost": "Support",
            "Seat Change Request": "Management",
            "Refund Request": "Billing",
            "Flight Cancellation": "Technical",
            "Meal Preference Change": "Management",
            "Overbooking Issue": "Management",
            "Missed Connecting Flight": "Support",
            "Damaged Baggage": "Support",
            "Check-in Issues": "Support",
            "Wrong Name on Ticket": "Management",
            "Duplicate Booking": "Technical",
            "Refund Delay": "Billing",
            "Upgrade Request": "Management",
            "Flight Rescheduling": "Technical"
        }

        self.issue_priority = {
            issue: priority for issue, priority in zip(self.departments.keys(), [2, 1, 3, 1, 2, 3, 2, 1, 2, 3, 2, 2, 1, 3, 2])
        }

        self.employee_credentials = {
            "support_emp1": {"password": "support123", "department": "Support"},
            "support_emp2": {"password": "support456", "department": "Support"},
            "billing_emp1": {"password": "billing123", "department": "Billing"},
            "management_emp1": {"password": "manage123", "department": "Management"},
            "management_emp2": {"password": "manage456", "department": "Management"},
            "technical_emp1": {"password": "tech123", "department": "Technical"},
            "technical_emp2": {"password": "tech456", "department": "Technical"},
        }

    def add_ticket(self, issue):
        if issue not in self.issue_priority:
            print("Invalid issue. Please select a valid issue.")
            return

        priority = self.issue_priority[issue]
        department = self.departments[issue]
        new_ticket = Ticket(self.ticket_counter, issue, priority, department=department)
        self.ticket_counter += 1

        if not self.head or self.head.priority > new_ticket.priority:
            new_ticket.next = self.head
            self.head = new_ticket
        else:
            temp = self.head
            while temp.next and temp.next.priority <= new_ticket.priority:
                temp = temp.next
            new_ticket.next = temp.next
            temp.next = new_ticket

        print(f"Ticket Added: ID={new_ticket.ticket_id}, Issue={new_ticket.issue}, Status={new_ticket.status}, Department={new_ticket.department}")

    def delete_ticket(self, ticket_id, department=None):
        temp = self.head
        prev = None
        while temp and temp.ticket_id != ticket_id:
            prev = temp
            temp = temp.next

        if not temp:
            print("Ticket ID not found.")
            return

        if department and temp.department != department:
            print("Unauthorized: Cannot delete tickets outside your department.")
            return

        if not prev:
            self.head = temp.next
        else:
            prev.next = temp.next

        self.deleted_tickets.append(temp)
        print(f"Deleted Ticket {ticket_id}.")

    def view_deleted_tickets_by_department(self, department):
        found = False
        print(f"\n---- Deleted Tickets for {department} Department ----")
        for ticket in self.deleted_tickets:
            if ticket.department == department:
                print(f"ID: {ticket.ticket_id}, Issue: {ticket.issue}, Status: {ticket.status}, Priority: {ticket.priority}")
                found = True
        if not found:
            print("No deleted tickets for your department.")
        print("--------------------------------------------")

    def update_ticket(self, ticket_id, new_status=None, department=None):
        temp = self.head
        while temp:
            if temp.ticket_id == ticket_id:
                if department and temp.department != department:
                    print("Unauthorized: Cannot update tickets outside your department.")
                    return
                if new_status in ["Open", "In Progress", "Resolved", "Closed"]:
                    temp.status = new_status
                    print(f"Updated Ticket {ticket_id}: Status={temp.status}")
                else:
                    print("Invalid status.")
                return
            temp = temp.next
        print("Ticket ID not found.")

    def display_tickets_by_department(self, department):
        temp = self.head
        found = False
        print(f"\n---- Tickets for {department} Department ----")
        while temp:
            if temp.department == department:
                print(f"ID: {temp.ticket_id}, Issue: {temp.issue}, Status: {temp.status}, Priority: {temp.priority}, Created At: {temp.creation_time}")
                found = True
            temp = temp.next
        if not found:
            print("No tickets found.")
        print("--------------------------------------------")

    def track_ticket(self, ticket_id, is_employee=False):
        temp = self.head
        while temp:
            if temp.ticket_id == ticket_id:
                visible_status = "Waiting for assignment" if temp.status.lower() == "unassigned" else temp.status
                if is_employee:
                    print(f"Ticket Status: ID={temp.ticket_id}, Issue={temp.issue}, Status={visible_status}, Department={temp.department}, Created At: {temp.creation_time}")
                else:
                    print(f"Ticket ID: {temp.ticket_id}, Issue: {temp.issue}, Status: {visible_status}")
                return
            temp = temp.next
        print("Ticket ID not found.")

    def authenticate_employee(self):
        username = input("Enter username: ").strip()
        password = input("Enter password: ").strip()

        if username in self.employee_credentials and self.employee_credentials[username]["password"] == password:
            print("Login successful.")
            return self.employee_credentials[username]["department"]
        else:
            print("Invalid credentials.")
            return None

    def menu(self):
        while True:
            print("\n1. Customer\n2. Employee\n3. Exit")
            role = input("Select role: ").strip()

            if role == '1':
                while True:
                    print("\nCustomer Menu:\n1. Add Ticket\n2. Delete Ticket\n3. Track Ticket\n4. Exit")
                    choice = input("Enter choice: ").strip()
                    if choice == '1':
                        print("Select an Issue:")
                        for i, issue in enumerate(self.issue_priority.keys(), 1):
                            print(f"{i}. {issue}")
                        try:
                            issue_choice = int(input("Enter issue number: ").strip())
                            issue_list = list(self.issue_priority.keys())
                            if 1 <= issue_choice <= len(issue_list):
                                self.add_ticket(issue_list[issue_choice - 1])
                            else:
                                print("Invalid choice.")
                        except ValueError:
                            print("Enter a valid number.")
                    elif choice == '2':
                        try:
                            ticket_id = int(input("Enter Ticket ID to delete: ").strip())
                            self.delete_ticket(ticket_id)
                        except ValueError:
                            print("Enter a valid Ticket ID.")
                    elif choice == '3':
                        try:
                            ticket_id = int(input("Enter Ticket ID to track: ").strip())
                            self.track_ticket(ticket_id)
                        except ValueError:
                            print("Enter a valid Ticket ID.")
                    elif choice == '4':
                        break
                    else:
                        print("Invalid choice.")

            elif role == '2':
                department = self.authenticate_employee()
                if not department:
                    continue

                while True:
                    print(f"\nEmployee Menu [{department} Department]:")
                    print("1. Display Tickets\n2. Update Ticket Status\n3. Delete Ticket\n4. View Deleted Tickets\n5. Logout")
                    choice = input("Enter choice: ").strip()
                    if choice == '1':
                        self.display_tickets_by_department(department)
                    elif choice == '2':
                        try:
                            ticket_id = int(input("Enter Ticket ID to update: ").strip())
                            temp = self.head
                            ticket = None
                            while temp:
                                if temp.ticket_id == ticket_id:
                                    ticket = temp
                                    break
                                temp = temp.next

                            if not ticket:
                                print("Ticket ID not found.")
                                continue
                            if ticket.department != department:
                                print("Unauthorized: This ticket does not belong to your department.")
                                continue

                            valid_statuses = ["Open", "In Progress", "Resolved", "Closed"]
                            while True:
                                new_status = input("Enter new status (Open, In Progress, Resolved, Closed): ").strip()
                                if new_status.lower() in [s.lower() for s in valid_statuses]:
                                    new_status = next(s for s in valid_statuses if s.lower() == new_status.lower())
                                    break
                                else:
                                    print("Invalid status entered. Please try again.")

                            self.update_ticket(ticket_id, new_status=new_status, department=department)
                        except ValueError:
                            print("Enter a valid Ticket ID.")
                    elif choice == '3':
                        try:
                            ticket_id = int(input("Enter Ticket ID to delete: ").strip())
                            self.delete_ticket(ticket_id, department=department)
                        except ValueError:
                            print("Enter a valid Ticket ID.")
                    elif choice == '4':
                        self.view_deleted_tickets_by_department(department)
                    elif choice == '5':
                        print("Logging out...")
                        break
                    else:
                        print("Invalid choice.")
            elif role == '3':
                print("Exiting system.")
                break
            else:
                print("Invalid choice. Try again.")

# Run the system
ticket_system = TicketSystem()
ticket_system.menu()



1. Customer
2. Employee
3. Exit


Select role:  1



Customer Menu:
1. Add Ticket
2. Delete Ticket
3. Track Ticket
4. Exit


Enter choice:  1


Select an Issue:
1. Flight Delay
2. Baggage Lost
3. Seat Change Request
4. Refund Request
5. Flight Cancellation
6. Meal Preference Change
7. Overbooking Issue
8. Missed Connecting Flight
9. Damaged Baggage
10. Check-in Issues
11. Wrong Name on Ticket
12. Duplicate Booking
13. Refund Delay
14. Upgrade Request
15. Flight Rescheduling


...

Ticket ID: 101, Issue: Flight Delay, Status: Waiting for assignment

Customer Menu:
1. Add Ticket
2. Delete Ticket
3. Track Ticket
4. Exit


Enter choice:  4



1. Customer
2. Employee
3. Exit


Select role:  3


Exiting system.
